In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests




/home/davidz/work/anaconda2/envs/carnd-advdl-odlab/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.4.0


/home/davidz/work/anaconda2/envs/carnd-advdl-odlab/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: No GPU found. Please use a GPU to train your neural network.
  import sys


In [3]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    input_image = graph.get_tensor_by_name(vgg_input_tensor_name)
    kp = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    pool3 = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    pool4 = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    pool7 = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)

    return input_image, kp, pool3, pool4, pool7
tests.test_load_vgg(load_vgg, tf)

Tests Passed


In [6]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
       :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    out = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, (1,1), padding='same',
                            kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3),
                            kernel_initializer = tf.contrib.layers.xavier_initializer())
    # pool7 upsample -> pool4 
    out = tf.layers.conv2d_transpose(out, num_classes, 4, (2,2), padding='same', 
                            kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3),
                            kernel_initializer = tf.contrib.layers.xavier_initializer())
    out = tf.layers.batch_normalization(out, training=True)

    conv4 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, (1,1), padding='same', 
                            kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3),
                            kernel_initializer = tf.contrib.layers.xavier_initializer())
    out = tf.add(out, conv4)

    # pool4 upsample -> pool3
    out = tf.layers.conv2d_transpose(out, num_classes, 4, (2,2), padding='same',
                            kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3),
                            kernel_initializer = tf.contrib.layers.xavier_initializer())
    out = tf.layers.batch_normalization(out, training=True)

    conv3 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, (1,1), padding='same', 
                            kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3),
                            kernel_initializer = tf.contrib.layers.xavier_initializer())
    out = tf.add(out, conv3)

    # pool4 upsample x8 -> original size 
    out = tf.layers.conv2d_transpose(out, num_classes, 16, (8,8), padding = 'same', 
                            kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3),
                            kernel_initializer = tf.contrib.layers.xavier_initializer())
    out = tf.layers.batch_normalization(out, training=True)

    return out
tests.test_layers(layers)

Tests Passed


In [9]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    correct_label = tf.reshape(correct_label, (-1, num_classes))
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=correct_label))
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss)
    
    return logits, train_op, cross_entropy_loss
tests.test_optimize(optimize)

Tests Passed


In [10]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        print("\n epoch = ", epoch, ": losses: ")
        for image, label in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss], feed_dict={input_image: image, 
                correct_label: label, keep_prob: 0.7, learning_rate: 0.001})
            print("{:.4f}, ".format(loss))

    pass
tests.test_train_nn(train_nn)


INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'


In [11]:
def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # parameters 
    epochs = 10
    batch_size = 1
    num_classes = 2

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network
        correct_label = tf.placeholder(tf.int32, [None, None, None, num_classes], name='correct_label')
        learning_rate = tf.placeholder(tf.float32, name='learning_rate')

        # TODO: Build NN using load_vgg, layers, and optimize function
        input_image, kp, p3, p4, p7 = load_vgg(sess, vgg_path)
        last_layer = layers(p3, p4, p7, num_classes)
        logits, train_op, cross_entropy_loss = optimize(last_layer, correct_label, learning_rate, num_classes)

        # TODO: Train NN using the train_nn function
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image, 
            correct_label, kp, learning_rate)

        # TODO: Save inference data using helper.save_inference_samples
        #  helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, kp, input_image)

        # OPTIONAL: Apply the trained model to a video

In [ ]:
run()

  4%|▎         | 35.2M/997M [00:09<03:34, 4.49MB/s]   /home/davidz/work/anaconda2/envs/carnd-advdl-odlab/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
997MB [04:34, 3.63MB/s]                            
